# Dataset

In [1]:
import numpy as np

# Exchange rate data.
tickers = ["USD", "EUR", "GBP", "CAD", "JPY", "CNY", "RUB", "MXN", "INR", "BRL"]
n = len(tickers)
F = np.zeros((n, n))
# USD
data = ([1.0, 0.87, 0.76, 1.31, 108.90, 6.72, 65.45, 19.11, 71.13, 3.69],
# EUR
[1.0, 0.88, 1.51, 125.15, 7.72, 75.23, 21.96, 81.85, 4.24],
# GBP
[1.0, 1.72, 142.94, 8.82, 85.90, 25.08, 93.50, 4.84],
# CAD
[1.0, 82.93, 5.11, 49.82, 14.54, 54.23, 2.81],
# JPY
[1.0, 0.062, 0.60, 0.18, 0.65, 0.034],
# CNY
[1.0, 9.74, 2.85, 10.61, 0.55],
# RUB
[1.0, 0.29, 1.09, 0.056],
# MXN
[1.0, 3.73, 0.19],
# INR
[1.0, 0.052],
# BRL
[1.0])
for i in range(n):
    F[i,i:] = data[i]
for j in range(n):
    for i in range(j+1,n):
        F[i,j] = 1.035/F[j,i]
        
# Initial and final portfolios.
c_req = np.arange(1,n+1)
c_req = 1e4*c_req/c_req.sum()
c_init = c_req[::-1]

# Problem solving

### Goal / Objective
We need to find the exchange matrix such the we minimize the cost of exchanges.

### Constraints
1. Given *c_init* which is initial amount of currenicies and we should not excced this while exchanging.
2. After currency exchanges each currency should not go below value given in *c_req*.
3. Exchanges should not be negative.
4. Currency i to currency i exchanges should be 0.

In [2]:
import cvxpy as cp

*X* is the control variable which is of size *(n, n)*. Where *n* is number of currencies. 

In [3]:
X = cp.Variable((n, n))

*USD_value_matrix* is a diagonal matrix which is helpful for computing current holdings in USD.

In [4]:
USD_value_matrix = np.array([np.sqrt(F[i, 0]/F[0, i]) for i in range(n)])
USD_value_matrix = np.diag(USD_value_matrix)

*c_final* is vector which indicates amount of each currency after exchanges are done.\
Mathematically it is,\
$$\vec{c\_final} = \vec{c\_init} - \mathbf{X}^T\vec{1} + \frac{\mathbf{X}}{\mathbf{F}}\vec{1}$$

In [5]:
ones = np.ones((n, 1))
c_init = c_init.reshape((n, 1))
c_req = c_req.reshape((n, 1))
out_exchanges = X.T@ones

c_final = c_init - out_exchanges + (X/F)@ones

$Initial\_holdings = \sum c_i\_init\sqrt{\frac{F_{i0}}{F_{0i}}}$, this represents total USD value of inital amount of currencies.\
$Final\_holdings = \sum c_i\_final\sqrt{\frac{F_{i0}}{F_{0i}}}$, this represents total USD value of final amount of currencies.\
Our objective is to minimize the cost of exchange which is,\
$$
Objective = min\{Initial\_holdings - Final\_holdings\}\\
= min\{\sum (c_i\_init - c_i\_final)\sqrt{\frac{F_{i0}}{F_{0i}}}\}\\
= min\{\vec{1} X \mathbf{USD\_value\_matrix} X \left(\vec{c\_init} - \vec{c\_final}\right)\}
$$

In [6]:
cost_of_exchange = ones.T@USD_value_matrix@(c_init - c_final)

Here constraints are,
1. $\mathbf{X} \ge \mathbf{0}$
2. $X_{ii} = 0$ $\forall i \in [0, n)$
3. $c\_final \ge c\_req$
4. $\mathbf{X}^T\vec{1} \le c\_init$

In [7]:
Constraints = [X >= 0,
              cp.diag(X) == 0,
              c_final >= c_req,
              out_exchanges <= c_init]

In [8]:
# Our objective is to minimize the total cost of exchange adhereing to above constraints
Objective = cp.Minimize(cost_of_exchange)
prob = cp.Problem(objective=Objective, constraints=Constraints) # Defining the problem
prob.solve(); # Solving the problem

In [9]:
X_star = X.value
print("X*: \n", np.round(X_star, 2))

X*: 
 [[  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.  ]
 [  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.  ]
 [  0.     0.     0.   545.45   0.     0.     0.     0.     0.     0.  ]
 [  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.  ]
 [  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.  ]
 [ 16.5    0.     0.     0.     0.     0.   727.27   0.     0.     0.  ]
 [  0.     0.     0.     0.     0.     0.     0.   369.09   0.     0.  ]
 [ 15.69   0.     0.     0.   181.82   0.     0.     0.     0.   181.82]
 [ 18.52   0.     0.     0.     0.     0.     0.     0.     0.     0.  ]
 [509.98   0.     0.     0.     0.     0.     0.     0.     0.     0.  ]]


In [10]:
# Priniting all the exchanges
for i in range(n):
    for j in range(n):
        if i != j:
            print("Amount of {} converted for {} is {}".format(tickers[j], tickers[i], X_star[i, j]))

Amount of EUR converted for USD is 4.2637896215846876e-09
Amount of GBP converted for USD is 3.6957171068345315e-09
Amount of CAD converted for USD is 5.646154722790683e-09
Amount of JPY converted for USD is 3.287106501755363e-07
Amount of CNY converted for USD is 1.6132108596088276e-08
Amount of RUB converted for USD is 1.5212464387640018e-07
Amount of MXN converted for USD is 4.4568050006253845e-08
Amount of INR converted for USD is 1.2890549246628384e-07
Amount of BRL converted for USD is 5.88965811318453e-09
Amount of USD converted for EUR is 5.059407609960219e-09
Amount of GBP converted for EUR is 8.669683794922768e-09
Amount of CAD converted for EUR is 2.1821708924757258e-08
Amount of JPY converted for EUR is 1.2551393210413878e-06
Amount of CNY converted for EUR is 3.3108335461217e-08
Amount of RUB converted for EUR is 3.093187791386473e-07
Amount of MXN converted for EUR is 8.835035610593377e-08
Amount of INR converted for EUR is 1.956952526260947e-07
Amount of BRL converted fo

In [11]:
# Printing the amount of each currency
print("Currency\t  Final\t\t Initial       Required")
print("-------------------------------------------------------")
for i in range(n):
    print("  {}\t\t{}\t{}\t{}".format(tickers[i], np.round(c_final[i, 0].value, 4),
                                        np.round(c_init[i, 0], 4), np.round(c_req[i, 0]), 4))

Currency	  Final		 Initial       Required
-------------------------------------------------------
  USD		1257.4958	1818.1818	182.0
  EUR		1636.3636	1636.3636	364.0
  GBP		1771.6702	1454.5455	545.0
  CAD		727.2727	1272.7273	727.0
  JPY		909.0909	1090.9091	909.0
  CNY		1090.9091	909.0909	1091.0
  RUB		1272.7273	727.2727	1273.0
  MXN		1454.5455	545.4545	1455.0
  INR		1636.3636	363.6364	1636.0
  BRL		1818.1818	181.8182	1818.0


In [12]:
# Printing cost of exchange
print("Total cost of exchange is {}".format(cost_of_exchange[0, 0].value))

Total cost of exchange is 7.720059340057901
